In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

%tensorflow_version 2.x

learning_rate = 0.1
train_steps = 5000

class MNISTDataset:
    """'Bare minimum' class to wrap MNIST numpy arrays into a dataset."""
    def __init__(self, train_imgs, train_lbs, test_imgs, test_lbls, batch_size,
                 to01=True, shuffle=True, seed=None,pad_bias=True):
        """
        Use seed optionally to always get the same shuffling (-> reproducible
        results).
        """
        self.num_samples,self.num_features = np.shape(train_imgs)
        self.batch_size = batch_size
        
        if not pad_bias:
          self.train_data = train_imgs
        else:
          self.train_data = np.c_[train_imgs , np.ones(self.num_samples)* 255]
        
        self.train_labels = train_lbs.astype(np.int32)
        self.test_data = test_imgs
        self.test_labels = test_lbls.astype(np.int32)

        if to01:
            # int in [0, 255] -> float in [0, 1]
            self.train_data = self.train_data.astype(np.float32) / 255
            self.test_data = self.test_data.astype(np.float32) / 255

        self.size = self.train_data.shape[0]

        if seed:
            np.random.seed(seed)
        if shuffle:
            self.shuffle_train()
        self.shuffle = shuffle
        self.current_pos = 0

    def next_batch(self):
        """Either gets the next batch, or optionally shuffles and starts a
        new epoch."""
        end_pos = self.current_pos + self.batch_size
        if end_pos < self.size:
            batch = (self.train_data[self.current_pos:end_pos],
                     self.train_labels[self.current_pos:end_pos])
            self.current_pos += self.batch_size
        else:
            # we return what's left (-> possibly smaller batch!) and prepare
            # the start of a new epoch
            batch = (self.train_data[self.current_pos:self.size],
                     self.train_labels[self.current_pos:self.size])
            if self.shuffle:
                self.shuffle_train()
            self.current_pos = 0
            print("Starting new epoch...")
        return batch

    def shuffle_train(self):
        shuffled_inds = np.arange(self.train_data.shape[0])
        np.random.shuffle(shuffled_inds)
        self.train_data = self.train_data[shuffled_inds]
        self.train_labels = self.train_labels[shuffled_inds]

fmnist_labels = {
0: "T-shirt/top",
1:	"Trouser",
2:	"Pullover",
3:	"Dress",
4:	"Coat",
5:	"Sandal",
6:	"Shirt",
7:	"Sneaker",
8:	"Bag",
9:	"Ankle boot"
}

#Fetch data
mnist =  tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

data = MNISTDataset(train_images.reshape([-1, 784]), train_labels, 
                    test_images.reshape([-1, 784]), test_labels,
                    batch_size=128,pad_bias=False, seed = 1807)

# Training the network
def random_initial_weight_matrix(x,y=None,epsilon=0.18,bias=False):
  
  if not bias:
    theta = (np.random.rand(x, y).astype(np.float32)* 2 * epsilon) - epsilon
    # theta = np.zeros([x,y],dtype=np.float32)
  else:
    # theta = np.zeros(x,dtype=np.float32)
    theta = (np.random.rand(x).astype(np.float32) * 2 * epsilon) - epsilon
  
  theta = tf.Variable(theta)
  return theta

riwm = lambda x,y : random_initial_weight_matrix(x,y)
ribm = lambda x,y : random_initial_weight_matrix(x,y,bias=True)

layer_config = {
    "in":784,
    "hidden":[200,50],
    "out": 10
}

W = [
     riwm(layer_config["in"],layer_config["hidden"][0]),
     riwm(layer_config["hidden"][0],layer_config["hidden"][1]),
     riwm(layer_config["hidden"][1],layer_config["out"])
    ]

b = [
     ribm(layer_config["hidden"][0],1),
     ribm(layer_config["hidden"][1],1),
     ribm(layer_config["out"],1)
    ]

record = {
    "train_cross_ent": [], 
    "weight": [
    ],
    "bias" : [

    ],
    "train_accuracy": []
}

for step in range(train_steps):
    img_batch, lbl_batch = data.next_batch()
    hyp = [
           [],
           [],
           []
    ]
    act = [
           [],
           [],
           []
    ]
    activation = [tf.nn.relu,tf.nn.relu,tf.nn.softmax]
    with tf.GradientTape() as tape:
        hyp[0] = tf.matmul(img_batch,W[0]) + b[0]
        act[0] = activation[0](hyp[0])

        hyp[1] = tf.matmul(act[0],W[1]) + b[1]
        act[1] = activation[1](hyp[1]) 

        hyp[2] = tf.matmul(act[1],W[2]) + b[2]
        act[2] = activation[2](hyp[2])

        xent = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = lbl_batch, logits = hyp[-1]))
        
        preds = tf.argmax(act[-1], axis=1, output_type=tf.int32)
        acc = tf.reduce_mean(tf.cast(tf.equal(preds, lbl_batch),
                             tf.float32))
        
        record["train_cross_ent"].append(xent)
        record["train_accuracy"].append(acc)
        # record["activation"][0].append(act[0])
        # record["activation"][1].append(act[1])
        # record["activation"][2].append(act[2])

    #backprop time   
    grads = tape.gradient(xent, [W[0], b[0], W[1], b[1], W[2], b[2]])
    
    W[0].assign_sub(learning_rate * grads[0])
    b[0].assign_sub(learning_rate * grads[1])

    W[1].assign_sub(learning_rate * grads[2])
    b[1].assign_sub(learning_rate * grads[3])
    
    W[2].assign_sub(learning_rate * grads[4])
    b[2].assign_sub(learning_rate * grads[5])

    record["weight"].append(W)
    record["bias"].append(b)


    if not step % 300:
        print("Loss: {} Accuracy: {}, step: {}".format(xent, acc,step))

def test_model(y_data,y_labels,_W,_b,_idx):
  # Load any of these two model weights, and do a forward pass to get test 
  #results
  # print("test examples, ",np.shape(data.test_data))
  # weights = best_model_weights[-1]
  assert len(y_data) == len(y_labels)
  _hyp = [
        [],
        [],
        []
  ]
  _act = [
        [],
        [],
        []
  ]
  _activation = [tf.nn.relu,tf.nn.relu,tf.nn.softmax]
  
  _hyp[0] = tf.matmul(y_data,_W[0])+ _b[0]
  _act[0] = _activation[0](_hyp[0])
  
  _hyp[1] = tf.matmul(_act[0],_W[1]) + _b[1]
  _act[1] = _activation[1](_hyp[1]) 

  _hyp[2] = tf.matmul(_act[1],_W[2]) + _b[2]
  _act[2] = _activation[2](_hyp[2])

  test_preds = tf.argmax(_act[2], axis=1,
                        output_type=tf.int32)
  test_preds = np.array(test_preds)

  test_acc = tf.reduce_mean(tf.cast(tf.equal(test_preds, y_labels),
                              tf.float32))
  result = f'Test accuracy {test_acc*100:.3f}% with model #{idx}'
  # print(f'Test accuracy {test_acc*100:.3f}% with model #{idx}')
  return test_preds,result


Loss: 2.6577274799346924 Accuracy: 0.1640625, step: 0
Loss: 0.6323590278625488 Accuracy: 0.765625, step: 300
Starting new epoch...
Loss: 0.5289076566696167 Accuracy: 0.8125, step: 600
Loss: 0.4538968503475189 Accuracy: 0.8671875, step: 900
Starting new epoch...
Loss: 0.41634881496429443 Accuracy: 0.828125, step: 1200
Starting new epoch...
Loss: 0.4013941287994385 Accuracy: 0.84375, step: 1500
Loss: 0.4404841363430023 Accuracy: 0.859375, step: 1800
Starting new epoch...
Loss: 0.33555006980895996 Accuracy: 0.890625, step: 2100
Starting new epoch...
Loss: 0.4095020890235901 Accuracy: 0.8359375, step: 2400
Loss: 0.2784830331802368 Accuracy: 0.8984375, step: 2700
Starting new epoch...
Loss: 0.37234389781951904 Accuracy: 0.8515625, step: 3000
Starting new epoch...
Loss: 0.32758259773254395 Accuracy: 0.875, step: 3300
Loss: 0.36581453680992126 Accuracy: 0.828125, step: 3600
Starting new epoch...
Loss: 0.19113998115062714 Accuracy: 0.953125, step: 3900
Loss: 0.3525354266166687 Accuracy: 0.9140

TypeError: ignored

In [0]:
W = record["weight"][-1]
idx = len(record["weight"]) - 1
b = record["bias"][idx]
preds,result = test_model(data.test_data,data.test_labels,W,b,idx)
print(result)
p_r_score = (precision_recall_fscore_support(data.test_labels,preds)[0:2])
print("\n")
for i in range(len(p_r_score[0])):
  print(f'{fmnist_labels[i]} : Precision {p_r_score[0][i]}, Recall {p_r_score[1][i]}')
print(classification_report(data.test_labels,preds,           # gives 0 p/r scores?
                          labels=list(fmnist_labels.keys()),target_names=[
                                                                          fmnist_labels[key] for key in fmnist_labels
                          ]))

Test accuracy 85.070% with model #4999


T-shirt/top : Precision 0.8452631578947368, Recall 0.803
Trouser : Precision 0.9907407407407407, Recall 0.963
Pullover : Precision 0.8249075215782984, Recall 0.669
Dress : Precision 0.8311444652908068, Recall 0.886
Coat : Precision 0.65, Recall 0.91
Sandal : Precision 0.9846153846153847, Recall 0.896
Shirt : Precision 0.7296587926509186, Recall 0.556
Sneaker : Precision 0.8117839607201309, Recall 0.992
Bag : Precision 0.941747572815534, Recall 0.97
Ankle boot : Precision 0.9828962371721779, Recall 0.862
              precision    recall  f1-score   support

 T-shirt/top       0.85      0.80      0.82      1000
     Trouser       0.99      0.96      0.98      1000
    Pullover       0.82      0.67      0.74      1000
       Dress       0.83      0.89      0.86      1000
        Coat       0.65      0.91      0.76      1000
      Sandal       0.98      0.90      0.94      1000
       Shirt       0.73      0.56      0.63      1000
     Sneaker     

In [0]:
fmnist_labels.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])